# Assignment 2: Bigram Language Model and Generative Pretrained Transformer (GPT)

Due Date Feb 4th, 2024 11:59pm

The objective of this assignment is to train a simplified transformer model. The primary differences between the implementation:
* tokenizer (we use a character level encoder simplicity and compute constraints)
* size (we are using 1 consumer grade gpu hosted on colab and a small dataset. in practice, the models are much larger and are trained on much more data)
* efficiency


Most modern LLMs have multiple training stages, so we won't get a model that is capable of replying to you yet. However, this is the first step towards a model like ChatGPT and Llama.




In [ ]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from torch import nn

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


## Part 1: Bigram MLP for TinyShakespeare (35 points)

1a) (1 point). Create a list `chars` that contains all unique characters in `text`

1b) (2 points). Implement `encode(s: str) -> list[int]`

1c) (2 points). Implement `decode(ids: list[int]) -> str`

1d) (5 points). Create two tensors, `inputs_one_hot` and `outputs_one_hot`. Use one hot encoding. Make sure to get every consecutive pair of characters. For example, for the word 'hello', we should create the following input-output pairs
```
he
el
ll
lo
```

1e) (10 points). Implement BigramOneHotMLP, a 2 layer MLP that predicts the next token. Specifically, implement the constructor, forward, and generate. The output dimension of the first layer should be 8. Use `torch.optim`. The activation function for the first layer should be `nn.LeakyReLU()`

Note: Use the `torch.nn.function.cross_entropy` loss. Read the [docs](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html) about how this loss function works. The logits are the output of a network WITHOUT an activation function applied to the last layer. There are activation functions are applied to every layer except the last.

1f) (5 points). Train the BigramOneHotMLP for 1000 steps.

1g) (5 points). Create two tensors, `input_ids` and `outputs_one_hot`. These `input_ids` will be used for the embedding layer.

1h) (5 points). Implement and train BigramEmbeddingMLP, a 2 layer mlp that predicts the next token. Specifically, implement the constructor, forward, and generate functions. The output dimension of the first layer should be 8. Use `torch.optim`.



Note: the output will look like gibberish


In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-05 00:09:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  6.98MB/s    in 0.2s    

2024-02-05 00:09:58 (6.98 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
# For the bigram model, let's use the first 1000 characters for the data

with open('input.txt', 'r') as f:
    text = f.read()
text = text[:1000]

In [ ]:

from torch.nn import functional as F
import torch.optim as optim
import torch.nn as nn




In [ ]:
#1a
#append unique characters from text to chars list
chars = sorted(set(text))



In [ ]:
#1b and 1c

#dictionary to create character to id mapping
chars2id = {c: i for i, c in enumerate(chars)}
#dictionary to create id to character mapping
id2chars = {i: c for c, i in chars2id.items()}

#encode functions returns a list of all the ids corresponding to characters in string
def encode(s: str) -> list[int]:
    return [chars2id[c] for c in text]

#decode function creates a string by getting all the characters corresponding to ids from the list returned by encode function
def decode(ids: list[int]) -> str:
    return ''.join([id2chars[i] for i in ids])




In [ ]:
#1d
#creates input and output tensors using one hot encoding which gives true values
def create_one_hot_inputs_and_outputs() -> list[torch.tensor, torch.tensor]:
    inputs_one_hot = []
    outputs_one_hot = []
    for c1, c2 in zip(text, text[1:]):
        inputs_one_hot.append(F.one_hot(torch.tensor(chars2id[c1]), len(chars)))
        outputs_one_hot.append(F.one_hot(torch.tensor(chars2id[c2]), len(chars)))
    return torch.stack(inputs_one_hot), torch.stack(outputs_one_hot)

inputs_one_hot, outputs_one_hot = create_one_hot_inputs_and_outputs()



In [ ]:
#1e
#using bigram model to train to predict output i.e next character based on input which is the start character
class BigramOneHotMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(chars), 8)
        self.activation = nn.LeakyReLU()
        self.fc2 = nn.Linear(8, len(chars))

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

    def generate(self, start='a', max_new_tokens=100) -> str:
        self.eval()
        with torch.no_grad():
            current_char = start
            word = current_char
            for _ in range(max_new_tokens):
                input_ids = F.one_hot(torch.tensor(chars2id[current_char]), len(chars)).unsqueeze(0).float()
                output = self(input_ids)
                next_char_id = torch.argmax(output).item()
                next_char = [k for k, v in chars2id.items() if v == next_char_id][0]
                current_char = next_char
                word += current_char
            return word


bigram_one_hot_mlp = BigramOneHotMLP()

generated_word = bigram_one_hot_mlp.generate()
print(f'Generated word: {generated_word}')



Generated word: akkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk


In [ ]:
#1f
learning_rate = 0.01
optimizer = optim.SGD(bigram_one_hot_mlp.parameters(), lr=learning_rate)

#using this to understand the cross entropy loss between the true values and predictions
# training loop
for epoch in range(1000):
    optimizer.zero_grad()
    predictions = bigram_one_hot_mlp(inputs_one_hot.float())
    loss = F.cross_entropy(predictions, outputs_one_hot.float())
    loss.backward()
    optimizer.step()
    #print(f'Epoch {epoch + 1}, Loss: {loss.item()}')
    # if epoch%100==0:
    #   print(f'Generated word:{bigram_one_hot_mlp.generate()}')
print(bigram_one_hot_mlp.generate())
print(len(bigram_one_hot_mlp.generate()))

a                                                                                                    
101


In [ ]:
#1g
#using embedding instead of one hot encoding for input to give true values
def create_embedding_inputs_and_outputs() -> list[torch.tensor, torch.tensor]:
    inputs_ids=[]
    outputs_one_hot=[]
    for c1, c2 in zip(text, text[1:]):
        inputs_ids.append(torch.tensor(chars2id[c1]))
        outputs_one_hot.append(F.one_hot(torch.tensor(chars2id[c2]), len(chars)))
    return torch.stack(inputs_ids), torch.stack(outputs_one_hot)


input_ids, outputs_one_hot = create_embedding_inputs_and_outputs()



In [ ]:
#1h
#using bigram model to train to predict output i.e next character based on input which is the start character but here using embedding for input
class BigramEmbeddingMLP(nn.Module):
    def __init__(self):
        # implement
        super().__init__()
        self.token_embedding=nn.Embedding(len(chars),8)
        self.fc1 = nn.Linear(8, 8)
        self.activation = nn.LeakyReLU()
        self.fc2 = nn.Linear(8, len(chars))

    def forward(self, x):
        # implement
        x = self.token_embedding(x)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

    def generate(self, start='a', max_new_tokens=100) -> str:
        # implement
        self.eval()
        with torch.no_grad():
            current_char = start
            word = current_char
            for _ in range(max_new_tokens):
                input_ids = torch.tensor(chars2id[current_char])
                output = self(input_ids)
                next_char_id = torch.argmax(output).item()
                next_char = [k for k, v in chars2id.items() if v == next_char_id][0]
                current_char = next_char
                word += current_char
            return word

bigram_embedding_mlp = BigramEmbeddingMLP()

generated_word = bigram_embedding_mlp.generate()
print(f'Generated word: {generated_word}')

learning_rate = 0.01
optimizer = optim.SGD(bigram_one_hot_mlp.parameters(), lr=learning_rate)

#using this to understand the cross entropy loss between the true values and predictions
# training loop
for _ in range(1000):
    # implement
    optimizer.zero_grad()
    predictions = bigram_one_hot_mlp(inputs_one_hot.float())
    loss = F.cross_entropy(predictions, outputs_one_hot.float())
    loss.backward()
    optimizer.step()


print(f'Generated word: {bigram_embedding_mlp.generate()}')

Generated word: aCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;i
Generated word: aCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;iCj;i


## Part 2: Generative Pretrained Transformer (65 points)

For this part, it is best to use a gpu. In the settings at the top go to Runtime -> Change Runtime Type and select T4 GPU

In [ ]:
# run nvidia-smi to check gpu usage
!nvidia-smi

Mon Feb  5 00:10:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              30W /  70W |    173MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# For the gpt model, let's use the full text

with open('input.txt', 'r') as f:
    text = f.read()

Implement a character level tokenization function.

1. Create a list of unique characters in the string. (1 points)
2. Implement a function `encode(s: str) -> list[int]` that takes a string and returns a list of ids (1 point)
3. Implement a function `decode(ids: list[int]) -> str` that takes a list of ids (ints) and returns a string (1 point)


In [ ]:
#1 Create a list of unique characters in the string.
chars = sorted(set(text))
print(len(chars))
chars2id={c:i for i,c in enumerate(chars)}
print(chars2id)
id2chars={i:c for c,i in chars2id.items()}

#2 Implement a function encode(s: str) -> list[int] that takes a string and returns a list of ids
def encode(s: str) -> list[int]:
    return [chars2id[c] for c in text]

#Implement a function decode(ids: list[int]) -> str that takes a list of ids (ints) and returns a string
def decode(ids: list[int]) -> str:
    return ''.join([id2chars[i] for i in ids])

65
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
block_size = 16
data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43])

To train a transformer, we feed the model `n` tokens (context) and try to predict the `n+1`th token (target) in the sequence.



In [ ]:
x = data[:block_size]
y = data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) the target: 64
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) the target: 43
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) the target: 52
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) the target: 10
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) the target: 0
when input is tensor([18, 47, 56, 57, 58,  1, 15, 

In [ ]:
#returns true values
batch_size = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def get_batch():
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x, y
print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14])
tensor([47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43])


### Single Self Attention Head (5 points)
![](https://i.ibb.co/GWR1XG0/head.png)

In [ ]:
import torch
from torch.nn import functional as F
import torch.optim as optim
import torch.nn as nn
import torch
from torch.nn import functional as F
import torch.optim as optim
import torch.nn as nn

# Define and initialize the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        #Linear projections for queries, keys, and values with head_size output dimensions
        self.q_proj = nn.Linear(64, head_size, bias=False)
        self.k_proj = nn.Linear(64, head_size, bias=False)
        self.v_proj = nn.Linear(64, head_size, bias=False)
        #dropout layer for regularization
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
       # Project queries, keys, and values
        B, T, C = x.shape
        k = self.k_proj(x)  # (B,T,C)
        q = self.q_proj(x)  # (B,T,C)
        #perform matrix multiplication between query and key transpose
        attention = q @ k.transpose(-2, -1) * C ** -0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)

        mask = torch.tril(torch.ones(T, T)).to(device)
        # Apply the mask to the attention scores
        masked_attention = attention.masked_fill(mask == 0, float('-inf'))
        # Apply softmax to get normalized attention weights
        masked_attention = F.softmax(masked_attention, dim=-1)  # (B, T, T)
        # Apply dropout to the attention weights
        masked_attention = self.dropout(masked_attention)

        # perform the weighted aggregation of the values
        v = self.v_proj(x)  # (B,T,C)
        #perform matrix multiplication between values and masked attention to get the output
        out = masked_attention @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

# Instantiate SelfAttentionHead with a head size of 16 and move it to the device
attn = SelfAttentionHead(16).to(device)
x = torch.randn((8, 32, 64)).float().to(device) #input tensor with shape (batch_size=8, sequence_length=32, input_size=64) and move it to the device
print(attn(x).shape)



torch.Size([8, 32, 16])


### Multihead Self Attention (5 points)

`constructor`

- Create 4 `SelfAttentionHead` instances. Consider using `nn.ModuleList`
- Create a linear layer with n_embd input dim and n_embd output dim

`forward`

In the forward implementation, pass `x` through each head, then concatenate all the outputs along the feature dimension, then pass the concatenated output through the linear layer

![](https://i.ibb.co/y5SwyZZ/multihead.png)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.linear = nn.Linear(num_heads*head_size,64)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        head_outputs = [head(x) for head in self.heads]
        concatenated = torch.cat(head_outputs, dim=-1)

        # Pass through linear layer
        linear_output = self.linear(concatenated)
        linear_output=self.dropout(linear_output)

        return linear_output
num_heads = 4 #initialising the number of heads to 4
head_size = 16


multi_head_attention = MultiHeadAttention(num_heads, head_size).to(device)

# Example usage
x = torch.randn((8, 32, 64)).float().to(device)
output = multi_head_attention(x)
print(output.shape)


torch.Size([8, 32, 64])


## MLP (2 points)
Implement a 2 layer MLP


![](https://i.ibb.co/C0DtrF5/ff.png)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, 256)
        self.activation = nn.LeakyReLU()
        self.fc2 = nn.Linear(256, 64)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x: torch.tensor) -> torch.tensor:
        x=self.fc1(x)
        x=self.activation(x)
        x=self.fc2(x)
        x=self.dropout(x)
        return x


x = torch.randn((8, 32, 64)).float()
mlp_model=MLP()
output = mlp_model(x)
print(output.shape)


torch.Size([8, 32, 64])


## Transformer block (20 points)

Layer normalization help training stability by normalizing the outputs of neurons within a single layer across all features for each individual data point, not across a full batch or a specific feature.

Dropout is a form of regularization to prevent overfitting.

This is the diagram of a transformer block:

![](https://i.ibb.co/X85C473/block.png)

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd: int, n_head: int):
        super().__init__()
        self.norm1 = nn.LayerNorm(n_embd)
        self.heads=MultiHeadAttention(n_head,16)
        self.fc1 = nn.Linear(n_embd, n_embd)



    def forward(self, x):
      y=self.norm1(x)
      y=self.heads(y)
      x=x+y
      z=self.norm1(x)
      z=self.fc1(z)
      x=x+z
      return x


x = torch.randn((8, 32, 64)).float().to(device)
tblock=Block(64,4).to(device)
output=tblock(x)
print(output.shape)

torch.Size([8, 32, 64])


## GPT

`constructor` (5 points)

1. create the token embedding table and the position embedding table
2. create variable `self.blocks` that is a series of `Block`s. The data will pass through each block sequentially. Consider using `nn.Sequential`
3. create a layer norm layer
4. create a linear layer for predicting the next token

`forward(self, idx, targets=None)`. (5 points)

`forward` takes a batch of context ids as input of size (B, T) and returns the logits and the loss, if targets is not None. If targets is None, return the logits and None.
1. get the token by using the token embedding table created in the constructor
2. create the position embeddings
3. sum the token and position embeddings to get the model input
4. pass the model through the blocks, the layernorm layer, and the final linear layer
5. compute the loss

`generate(start_char, max_new_tokens, top_p, top_k, temperature) -> str` (5 points)
1. implement top p, top_k, and temperature for sampling



![](https://i.ibb.co/n8sbQ0V/Screenshot-2024-01-23-at-8-59-08-PM.png)

In [ ]:
import numpy as np
class GPT(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        # create the token embedding table
        self.token_embedding = nn.Embedding(len(chars),n_embd)

        # Create the position embedding table
        self.positional_embedding = nn.Embedding(32, n_embd)
        # Create dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Create blocks using nn.Sequential to go through series of 4 blocks
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(4)])

        # Create a layer norm layer
        self.layer_norm = nn.LayerNorm(n_embd)

        # Create a linear layer for predicting the next token
        self.linear = nn.Linear(n_embd, len(chars))


    def forward(self, idx, targets=None):
        B,T=idx.shape
        # print(idx)
        # print(B)
        # print(T)
        # print(len(chars))
        # print(f"Min index: {torch.min(idx)}, Max index: {torch.max(idx)}")
        # Create token emebeddings for the index passed from generate method
        token_embeddings = self.token_embedding(idx).to(device)


        # print(idx.size(1))

        # Create position embeddings
        positional_embedding = self.positional_embedding(torch.stack([torch.arange(T) for _ in range(B)], 0).to(device))
        # print(token_embeddings.shape)
        # print(positional_embedding.shape)

        # Sum token and position embeddings
        x = token_embeddings + positional_embedding
        #pass the embedded output through dropout layer
        x = self.dropout(x)
        # print(x.size())
        # Pass through blocks
        x= self.blocks(x)


        # Apply layer norm
        x = self.layer_norm(x)

        # Linear layer for predicting the next token
        logits = self.linear(x)

        #return the loss and logits
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits,loss


    def generate(self, start_char, max_new_tokens, top_p, top_k, temperature):
        # Initialize the input sequence with the start character
        self.eval()
        with torch.no_grad():
          # id corresponding to the start character as tensor
            current_seq = torch.tensor([chars2id[start_char]], dtype=torch.long).to(device).unsqueeze(0)
            # print(current_seq)
            for _ in range(max_new_tokens):
                """pass the last 32 elements from the tensor as 32 is the sequence length
                for selecting a portion of the input sequence"""
                logits, loss = self(current_seq[:, -32:])
                # print("Logits shape:", logits.shape)
                # print(logits)

                # Apply temperature to logits for diversity
                logits = logits[:, -1, :]
                # print(logits)
                scaled_logits = logits / temperature
                # Apply softmax to get probabilities
                probabilities = F.softmax(scaled_logits, dim=-1)

                # Apply top-k or top-p nucleus sampling
                if top_k is not None:
                    sampled_index = top_k_sampling(probabilities, k=top_k)
                elif top_p is not None:
                    sampled_index = top_p_sampling(probabilities, p=top_p)


                # Convert the sampled index to a PyTorch tensor
                sampled_token = torch.tensor([[sampled_index]]).to(device)
                # print(sampled_token)

                # Append the sampled token to the sequence
                current_seq = torch.cat([current_seq, sampled_token], dim=1)

        #print("Loss",loss)
        # Convert the generated sequence to a string using ids2character mapping
        generated_string = ''.join([id2chars[token.item()] for token in current_seq[0]])
        return generated_string

def top_k_sampling(probabilities, k=5):
        """
        Performs top-k sampling from a probability distribution.

        :param probabilities: A 1D numpy array containing the probability distribution.
        :param k: The number of top elements to consider for sampling.
        :return: An index sampled from the top-k distribution.
        """
        probabilities = probabilities.cpu().numpy().flatten()
      # Get indices of the top k probabilities
        top_k_indices = np.argsort(probabilities)[-k:]

      # Extract the top k probabilities
        top_k_probabilities = probabilities[top_k_indices]

      # Normalize the top k probabilities so they sum to 1
        top_k_probabilities /= top_k_probabilities.sum()

      # Sample from the top k elements
        chosen_index = np.random.choice(top_k_indices, p=top_k_probabilities)

        return chosen_index

def top_p_sampling(probabilities, p=0.9):
    """
    Selects tokens from a probability distribution that have a cumulative probability
    greater than the threshold p.

    :param probabilities: A 1D numpy array containing the probability distribution.
    :param p: The cumulative probability threshold (0 < p <= 1).
    :return: An index sampled according to the top-p distribution.
    """
    probabilities = probabilities.cpu().numpy().flatten()
    # print(f"{probabilities=}")
    # Sort probabilities in descending order and also get the original indices
    if len(probabilities) > 1:
      sorted_indices = np.argsort(probabilities)[::-1]
    else:
    # Handle the case where probabilities is empty or has length 1
      sorted_indices = np.argsort(probabilities)
    # print(f"{sorted_indices=}")

    sorted_probabilities = probabilities[sorted_indices]

    # print(f"{sorted_probabilities=}")

    # Compute cumulative probabilities
    cumulative_probabilities = np.cumsum(sorted_probabilities)
    # print(f"{cumulative_probabilities=}")

    # Find the threshold index
    cutoff_index = np.where(cumulative_probabilities > p)[0][0]
    # print(f"{cutoff_index=}")

    # Filter out indices and probabilities that don't meet the threshold
    filtered_indices = sorted_indices[:cutoff_index + 1]
    # print(f"{filtered_indices=}")
    filtered_probabilities = sorted_probabilities[:cutoff_index + 1]
    # print(f"{filtered_probabilities=}")

    # Normalize the filtered probabilities
    filtered_probabilities /= filtered_probabilities.sum()
    # print(f"normalized_filtered_probabilities{filtered_probabilities}")

    # Sample from the filtered distribution
    chosen_index = np.random.choice(filtered_indices, p=filtered_probabilities)

    return chosen_index








gpt_model = GPT(64, 4).to(device)
# Correcting the variable name for model in the generate method
generated_text = gpt_model.generate(start_char='a', max_new_tokens=100,top_k=None,top_p=0.9, temperature=1.0)
print(generated_text)


azvt Py$?pBGF Bjts-l&
Yr,OpK!uXUCWS;v!kLCU!Jcz$q
mWtKEMONUU?
Px?KkHNr$$Fm!;BS-EwKrmtaKe-DhrMpjjjMU.&H


### Training loop (15 points)

implement training loop

In [ ]:
gpt_model = GPT(64, 4).to(device)
# make you are running this on the GPU
max_iters = 10000
learning_rate = 0.001
optimizer = optim.SGD(gpt_model.parameters(), lr=learning_rate)
for iter in range(max_iters):
  optimizer.zero_grad()
# sample a batch of data
  xb, yb = get_batch()
# evaluate the loss
  logits, loss = gpt_model(xb, yb)
  loss.backward()
  optimizer.step()
  if iter%1000==0:
      print(f'Iteration {iter+1}, Loss: {loss.item()}')
      print(f'Generated text:{gpt_model.generate(start_char="a", max_new_tokens=100,top_k=5,top_p=None, temperature=1.0)}')


Iteration 1, Loss: 4.3076491355896
Generated text:ayKDjbVmwg!&GllgBf'oXI'z'td:P.gWoFPa3,HxKHEHHHHHozz!NKHxKHEHxIoz!NxEiziHKHHHxIHHxKz!:PaaobHHEnFPcxKz,
Iteration 1001, Loss: 3.5863616466522217
Generated text:ar t t  he    t leaoehhy    cenhhh a  c a  aot  t aoeo a c  cai too a co cot eha aihenhhoh a ahhea   
Iteration 2001, Loss: 3.2973084449768066
Generated text:anith   sto e terest ee  tthe t t oheotooo  aoe oth t  eo thao  oth teo e  o ee  a ootheaaaao   eoo t
Iteration 3001, Loss: 3.1949102878570557
Generated text:anhe t t  toe t e thanthe stoeo  t the  a  othenit aaosa t o t te  a aen sea  oha toothheaaa a s to s
Iteration 4001, Loss: 3.0600011348724365
Generated text:at te t s t   se  t t e aenaee aos t t aoo  aa a aaa o a ao othaotis a o os arisa anot t sa osea the 
Iteration 5001, Loss: 3.0313384532928467
Generated text:aroor t th eaouer tor seeareae t too  o an tenhe an a teonos  a i t t th the the a  t iaa sooshaa t s
Iteration 6001, Loss: 3.0814859867095947
Generated text:at

### Generate text


print some text that your model generates

In [ ]:
print(gpt_model.generate(start_char="a", max_new_tokens=100,top_k=5,top_p=None, temperature=1.0))

ane serean ther s wn t wieriat, c tharan ina t ar isaande, isares hanind at w thon thare, hisa tothar
